In [ ]:
from sklearn.metrics import roc_auc_score, classification_report, roc_curve, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.preprocessing.sequence import TimeseriesGenerator
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow.keras.layers import LSTM, Dense
#from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scipy.signal import butter, filtfilt
from keras.layers import TimeDistributed
from keras.models import Sequential
from sklearn.utils import shuffle
from keras.layers import Dropout
import matplotlib.pyplot as plt
import tensorflow as tf
import os.path as path
import seaborn as sns
import pandas as pd
import numpy as np
import optuna
import scipy
import os
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
path = path.abspath(path.join(os.getcwd(),"../"))
path

In [ ]:
filename = path + '\\new\Healthy1.csv'
d1 = pd.read_csv(filename, header=0)

filename = path + '\\new\Healthy2.csv'
d2 = pd.read_csv(filename, header=0)

filename = path + '\\new\PAD1.csv'
d3 = pd.read_csv(filename, header=0)

filename = path + '\\new\PAD2.csv'
d4 = pd.read_csv(filename, header=0)

filename = path + '\\new\PAD3.csv'
d5 = pd.read_csv(filename, header=0)

In [ ]:
d2.sort_index(inplace=True)
d2.interpolate(method='spline', order=3, inplace=True)
d2.ffill(inplace=True)  # fill NAs at the beginning of the column
d2.bfill(inplace=True)  # fill NAs at the end of the column

In [ ]:
d4.sort_index(inplace=True)
d4.interpolate(method='spline', order=3, inplace=True)
d4.ffill(inplace=True)  # fill NAs at the beginning of the column
d4.bfill(inplace=True)  # fill NAs at the end of the column

In [ ]:
d5.sort_index(inplace=True)
d5.interpolate(method='spline', order=3, inplace=True)
d5.ffill(inplace=True)  # fill NAs at the beginning of the column
d5.bfill(inplace=True)  # fill NAs at the end of the column

In [ ]:
df = d1.append(d2)
df = df.append(d3)
df = df.append(d4)
df = df.append(d5)
print(df.shape)

In [ ]:
df.head(2)

In [ ]:
df['filename'].value_counts().max()

## Extracting acceleration from position

In [ ]:
acc_df = pd.DataFrame()

pos_cols = ['R.ASIS_x', 'R.ASIS_y', 'R.ASIS_z', 'Right_PSIS_x',
       'Right_PSIS_y', 'Right_PSIS_z', 'L.ASIS_x', 'L.ASIS_y', 'L.ASIS_z',
       'Left_PSIS_x', 'Left_PSIS_y', 'Left_PSIS_z', 'V.Sacral_x', 'V.Sacral_y',
       'V.Sacral_z', 'Right_Troch_x', 'Right_Troch_y', 'Right_Troch_z',
       'Right_mid_thigh_x', 'Right_mid_thigh_y', 'Right_mid_thigh_z',
       'R.Thigh_x', 'R.Thigh_y', 'R.Thigh_z', 'R.Knee_x', 'R.Knee_y',
       'R.Knee_z', 'R.Shank_x', 'R.Shank_y', 'R.Shank_z',
       'Right_lower_shank_x', 'Right_lower_shank_y', 'Right_lower_shank_z',
       'R.Ankle_x', 'R.Ankle_y', 'R.Ankle_z', 'R.Toe_x', 'R.Toe_y', 'R.Toe_z',
       'R.Heel_x', 'R.Heel_y', 'R.Heel_z', 'Right_MTP_lat_x',
       'Right_MTP_lat_y', 'Right_MTP_lat_z', 'Right_cal_lat_x',
       'Right_cal_lat_y', 'Right_cal_lat_z', 'Left_Troch_x', 'Left_Troch_y',
       'Left_Troch_z', 'Left_mid_thigh_x', 'Left_mid_thigh_y',
       'Left_mid_thigh_z', 'L.Thigh_x', 'L.Thigh_y', 'L.Thigh_z', 'L.Knee_x',
       'L.Knee_y', 'L.Knee_z', 'L.Shank_x', 'L.Shank_y', 'L.Shank_z',
       'Left_lower_shank_x', 'Left_lower_shank_y', 'Left_lower_shank_z',
       'L.Ankle_x', 'L.Ankle_y', 'L.Ankle_z', 'L.Toe_x', 'L.Toe_y', 'L.Toe_z',
       'L.Heel_x', 'L.Heel_y', 'L.Heel_z', 'Left_MTP_lat_x', 'Left_MTP_lat_y',
       'Left_MTP_lat_z', 'Left_cal_lat_x', 'Left_cal_lat_y', 'Left_cal_lat_z']


for col in pos_cols:
        pos = df[col].to_numpy()
        time = df['Time'].to_numpy()
        vel = np.gradient(pos, time)
        acc = np.gradient(vel, time)
        acc_df[f'acc_{col}'] = acc
        
acc_df['filename'] = df['filename'].values
acc_df['PAD'] = df['PAD'].values
acc_df['Time'] = df['Time'].values

In [ ]:
acc_df.shape

In [ ]:
acc_df.head(2)

In [ ]:
data_subset = df[['Time', 'R.Heel_y']].iloc[0:200]
plt.figure(figsize=(10, 3))
plt.plot(data_subset['Time'], data_subset['R.Heel_y'], label='R.Heel_y', linewidth=2, color='r')
#plt.title('Time-series Plot of R.Heel_x')
plt.xlabel('Time (s)')
plt.ylabel('Value')
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

In [ ]:
coefficients = np.polyfit(time, signal, 1)
polynomial = np.poly1d(coefficients)
drift = polynomial(time)

# Remove the drift
corrected_signal = signal - drift

# Plot
plt.figure()
plt.subplot(3, 1, 1)
plt.title('Original Signal')
plt.plot(time, signal)

plt.subplot(3, 1, 2)
plt.title('Fitted Drift')
plt.plot(time, drift)

plt.subplot(3, 1, 3)
plt.title('Corrected Signal')
plt.plot(time, corrected_signal)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.title("Original and Corrected Signal")
plt.plot(time, signal, label='Original Signal', color='blue')
plt.plot(time, corrected_signal, label='Corrected Signal', color='red')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

In [ ]:
acc_df.head()

In [ ]:
#acc_df['Time'] = df['Time']
data_subset = acc_df[['time', 'acc_R.Heel_y']].iloc[0:200]
plt.figure(figsize=(10, 3))
plt.plot(data_subset['time'], data_subset['acc_R.Heel_y'], label='acc_R.Heel_y', linewidth=2, color='b')
#plt.title('Time-series Plot of R.Heel_x')
plt.xlabel('Time (s)')
plt.ylabel('Value')
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

In [ ]:
acc_df['Subject'] = acc_df['filename'].str.replace('p1c1_B1\.trc$', '', regex=True).str.replace('p1c1_B1\.csv$', '', regex=True).str.replace('c1p1_B1\.csv$', '', regex=True).str.replace('c1p1_B1\.trc$', '', regex=True).str.replace('p1c1_B01\.csv$', '', regex=True).str.replace('p1c1_B11\.csv$', '', regex=True)
acc_df.Subject.nunique()

In [ ]:
acc_df.head()

In [ ]:
# List of features
features = ['acc_R.ASIS', 'acc_Right_PSIS', 'acc_L.ASIS', 'acc_Left_PSIS', 'acc_V.Sacral', 'acc_Right_Troch'
       ,'acc_Right_mid_thigh','acc_R.Thigh', 'acc_R.Knee', 'acc_R.Shank', 'acc_Right_lower_shank', 'acc_R.Ankle'
       , 'acc_R.Toe','acc_R.Heel', 'acc_Right_MTP_lat', 'acc_Right_cal_lat', 'acc_Left_Troch', 'acc_Left_mid_thigh'
       , 'acc_L.Thigh', 'acc_L.Knee', 'acc_L.Shank', 'acc_Left_lower_shank', 'acc_L.Ankle', 'acc_L.Toe', 'acc_L.Heel'
       , 'acc_Left_MTP_lat', 'acc_Left_cal_lat']

In [ ]:
n_features = len(features)
n_cols = 2  # Number of subplots per row
n_rows = int(np.ceil(n_features / n_cols))

# Create a new figure and axis array
fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(8 * n_cols, 6 * n_rows))

# Flatten the axis array and remove extra subplots
axs = axs.flatten()
for ax in axs[n_features:]:
    ax.remove()

# Loop through each feature
for idx, feature in enumerate(features):
    ax = axs[idx]
    
    # Get the data for the current feature and slice the first 200 samples
    feature_data = acc_df[[f"{feature}_x", f"{feature}_y", f"{feature}_z"]][:200]
    
    # Normalize the data
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(feature_data)
    normalized_data = pd.DataFrame(normalized_data, columns=feature_data.columns, index=feature_data.index)
    
    # Plot each dimension (x, y, z)
    for dimension in ['x', 'y', 'z']:
        ax.plot(normalized_data.index, normalized_data[f"{feature}_{dimension}"], label=dimension)
    
    # Set the title and labels
    ax.set_title(f'{feature} - Dimensions (Normalized)')
    ax.set_xlabel('Time')
    ax.set_ylabel('Normalized Value')
    
    # Add a legend
    ax.legend()

# Show the plot
plt.tight_layout()
plt.savefig("acc.pdf", format='pdf', bbox_inches='tight')
plt.show()


In [ ]:
features = ['R.ASIS', 'Right_PSIS', 'L.ASIS', 'Left_PSIS', 'V.Sacral', 'Right_Troch'
       ,'Right_mid_thigh','R.Thigh', 'R.Knee', 'R.Shank', 'Right_lower_shank', 'R.Ankle'
       , 'R.Toe','R.Heel', 'Right_MTP_lat', 'Right_cal_lat', 'Left_Troch', 'Left_mid_thigh'
       , 'L.Thigh', 'L.Knee', 'L.Shank', 'Left_lower_shank', 'L.Ankle', 'L.Toe', 'L.Heel'
       , 'Left_MTP_lat', 'Left_cal_lat']

In [ ]:
n_features = len(features)
n_cols = 2  # Number of subplots per row
n_rows = int(np.ceil(n_features / n_cols))

# Create a new figure and axis array
fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(8 * n_cols, 6 * n_rows))

# Flatten the axis array and remove extra subplots
axs = axs.flatten()
for ax in axs[n_features:]:
    ax.remove()

# Loop through each feature
for idx, feature in enumerate(features):
    ax = axs[idx]
    
    # Get the data for the current feature and slice the first 200 samples
    feature_data = df[[f"{feature}_x", f"{feature}_y", f"{feature}_z"]][:200]
    
    # Normalize the data
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(feature_data)
    normalized_data = pd.DataFrame(normalized_data, columns=feature_data.columns, index=feature_data.index)
    
    # Plot each dimension (x, y, z)
    for dimension in ['x', 'y', 'z']:
        ax.plot(normalized_data.index, normalized_data[f"{feature}_{dimension}"], label=dimension)
    
    # Set the title and labels
    ax.set_title(f'{feature} - Dimensions (Normalized)')
    ax.set_xlabel('Time')
    ax.set_ylabel('Normalized Value')
    
    # Add a legend
    ax.legend()

# Show the plot
plt.tight_layout()
plt.savefig("motion.pdf", format='pdf', bbox_inches='tight')
plt.show()


In [ ]:
acc_df2 = acc_df[['Time', 'PAD', 'Subject']]

## Filtering acceleration using 4th order butterworth 20 Hz.

In [ ]:
# Define the filter parameters
cutoff_freq = 20 # Hz
order = 4

# Define the filter function
def butterworth_filter(data, cutoff_freq, order, sampling_rate):
    nyquist_freq = 0.5 * sampling_rate
    normal_cutoff = cutoff_freq / nyquist_freq
    b, a = butter(order, normal_cutoff, btype='lowpass', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

# Loop through each column (except the first two which are characters)
for col in df.columns[1:-2]:
    # Apply the Butterworth filter to the current column
    filtered_data = butterworth_filter(df[col], cutoff_freq, order, 60)
    # Replace the original data with the filtered data
    df[col] = filtered_data

In [ ]:
df.head()

In [ ]:
# Assuming you have a list of features named 'features'
features = ['acc_R.ASIS', 'acc_Right_PSIS', 'acc_Right_PSIS', 'acc_Right_PSIS', 'acc_L.ASIS', 'acc_L.ASIS'
            , 'acc_V.Sacral', 'acc_V.Sacral', 'acc_V.Sacral', 'acc_L.ASIS', 'acc_Left_PSIS', 'acc_Left_PSIS', 'acc_Left_PSIS', 'acc_Right_Troch'
            , 'acc_Right_Troch', 'acc_Right_Troch', 'acc_Right_mid_thigh', 'acc_Right_mid_thigh'
            , 'acc_Right_mid_thigh', 'acc_R.Thigh', 'acc_R.Thigh', 'acc_R.Thigh', 'acc_R.Knee'
            , 'acc_R.Knee', 'acc_R.Knee', 'acc_R.Shank', 'acc_R.Shank', 'acc_R.Shank', 'acc_Right_lower_shank'
            , 'acc_Right_lower_shank', 'acc_Right_lower_shank', 'acc_R.Ankle', 'acc_R.Toe', 'acc_R.Toe'
            , 'acc_R.Toe', 'acc_R.Heel', 'acc_R.Heel', 'acc_R.Heel', 'acc_Right_MTP_lat', 'acc_Right_cal_lat'
            , 'acc_Left_Troch', 'acc_Left_Troch', 'acc_Left_Troch', 'acc_Left_mid_thigh', 'acc_L.Thigh'
            , 'acc_L.Thigh', 'acc_L.Knee', 'acc_Left_lower_shank', 'acc_Left_lower_shank', 'acc_L.Ankle'
            , 'acc_L.Ankle', 'acc_L.Toe', 'acc_L.Toe', 'acc_L.Toe', 'acc_L.Heel', 'acc_L.Heel'
            , 'acc_Left_MTP_lat', 'acc_Left_MTP_lat', 'acc_Left_cal_lat', 'acc_Left_cal_lat']

cutoff_freq = 15  # Hz
order = 4
sampling_rate = 60  # Assuming the sampling rate is 60 Hz, change it to the actual value


#df.sort_index().interpolate(method='spline', order=3, inplace=True)
    
    
filtered_data_list = []  # Create an empty list to store the filtered data

for feature in features:
    # Get the data for the current feature
    feature_data = acc_df[[f"{feature}_x", f"{feature}_y", f"{feature}_z"]]

    # Normalize the data
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(feature_data)
    normalized_data = pd.DataFrame(normalized_data, columns=feature_data.columns, index=feature_data.index)

    # Create a new DataFrame to store the filtered data for this feature
    filtered_feature = pd.DataFrame()

    for dimension in ['x', 'y', 'z']:
        data = normalized_data[f"{feature}_{dimension}"]
        filtered_data = butterworth_filter(data, cutoff_freq, order, sampling_rate)
        filtered_feature[f"{feature}_{dimension}"] = filtered_data

    # Store the filtered data for the current feature in the list
    filtered_data_list.append(filtered_feature)

# Combine the filtered data for all features into a single DataFrame
filtered_df = pd.concat(filtered_data_list, axis=1)

In [ ]:
target_column = 'acc_V.Sacral_x'

# Calculate the correlation between the target column and all other columns
correlations = filtered_df.corr()[target_column]

# Print the correlations
high_correlation_features = correlations[correlations > 0.8].index.tolist()

# Print the list of features with > 0.8 correlation
print(high_correlation_features)

In [ ]:
#dr1 = ['R.ASIS_x', 'Right_PSIS_x', 'Left_PSIS_x', 'Right_Troch_x', 'Right_mid_thigh_x', 'R.Knee_x', 'R.Shank_x', 'Right_lower_shank_x', 'R.Ankle_x']
dr2 = ['R.ASIS_y', 'Right_PSIS_y', 'L.ASIS_y', 'Left_PSIS_y', 'Right_Troch_y', 'Right_mid_thigh_y', 'R.Thigh_y', 'R.Knee_y', 'R.Knee_z', 'R.Shank_y', 'Right_lower_shank_y', 'R.Ankle_y', 'R.Toe_y', 'R.Heel_y', 'Right_MTP_lat_y', 'Right_cal_lat_y', 'Left_Troch_y', 'L.Shank_y', 'Left_MTP_lat_y', 'Left_cal_lat_y']
dr3 = ['Right_PSIS_z', 'Left_PSIS_z']
a = dr2 + dr3

In [ ]:
filtered_df = filtered_df.drop(a, axis=1)

In [ ]:
filtered_df.head()

In [ ]:
correlation_matrix = filtered_df.corr()

# Function to get the first pair of highly correlated columns
def get_first_highly_correlated_pair(corr_matrix, threshold=0.8):
    # Upper triangle of correlations (to avoid duplicate pairs)
    upper_triangle = np.triu(corr_matrix, k=1)
    # Find row, col index for the first occurrence of correlation greater than threshold
    row, col = np.where(np.abs(upper_triangle) > threshold)
    
    if row.size > 0:
        # Return names of the first pair of highly correlated features
        return corr_matrix.columns[row[0]], corr_matrix.columns[col[0]]
    else:
        return None, None

cols_to_drop = []

while True:
    # Get the first pair of highly correlated features
    col1, col2 = get_first_highly_correlated_pair(correlation_matrix)
    
    # If no pair found, break out of loop
    if not col1:
        break

    # Drop one of the columns from the dataframe and the correlation matrix
    filtered_df.drop(col2, axis=1, inplace=True)
    correlation_matrix.drop([col2], axis=1, inplace=True)
    correlation_matrix.drop([col2], axis=0, inplace=True)

    # Keep track of dropped columns
    cols_to_drop.append(col2)

In [ ]:
filtered_df.shape

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))

# Generate a custom diverging colormap: Highly correlated in red, less correlated in blue
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
# Here, we consider correlations > 0.7 as high, you can adjust this threshold as needed
sns.heatmap(correlation_matrix, cmap=cmap, vmax=.8, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=False)

plt.title('Feature Correlation Heatmap')
plt.savefig("correlation_heatmap.jpg", format='jpg', dpi=300)
plt.show()

In [ ]:
print(list(filtered_df.columns))

In [ ]:
# Redo the filtering for the selected features without scaling to avoid data leakage
# Define the filter function
def butterworth_filter(data, cutoff_freq, order, sampling_rate):
    nyquist_freq = 0.5 * sampling_rate
    normal_cutoff = cutoff_freq / nyquist_freq
    b, a = butter(order, normal_cutoff, btype='lowpass', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

# Assuming you have a list of features named 'features'
features = ['acc_R.ASIS_x', 'acc_Right_PSIS_x', 'acc_Right_PSIS_y', 'acc_Right_PSIS_z', 'acc_L.ASIS_x', 'acc_L.ASIS_y'
            , 'acc_L.ASIS_z', 'acc_Left_PSIS_x', 'acc_Left_PSIS_y', 'acc_Left_PSIS_z', 'acc_Right_Troch_x'
            , 'acc_Right_Troch_y', 'acc_Right_Troch_z', 'acc_Right_mid_thigh_x', 'acc_Right_mid_thigh_y'
            , 'acc_Right_mid_thigh_z', 'acc_R.Thigh_x', 'acc_R.Thigh_y', 'acc_R.Thigh_z', 'acc_R.Knee_x'
            , 'acc_R.Knee_y', 'acc_R.Knee_z', 'acc_R.Shank_x', 'acc_R.Shank_y', 'acc_R.Shank_z', 'acc_Right_lower_shank_x'
            , 'acc_Right_lower_shank_y', 'acc_Right_lower_shank_z', 'acc_R.Ankle_y', 'acc_R.Toe_x', 'acc_R.Toe_y'
            , 'acc_R.Toe_z', 'acc_R.Heel_x', 'acc_R.Heel_y', 'acc_R.Heel_z', 'acc_Right_MTP_lat_y', 'acc_Right_cal_lat_y'
            , 'acc_Left_Troch_x', 'acc_Left_Troch_y', 'acc_Left_Troch_z', 'acc_Left_mid_thigh_x', 'acc_L.Thigh_y'
            , 'acc_L.Thigh_z', 'acc_L.Knee_y', 'acc_Left_lower_shank_y', 'acc_Left_lower_shank_z', 'acc_L.Ankle_x'
            , 'acc_L.Ankle_y', 'acc_L.Toe_x', 'acc_L.Toe_y', 'acc_L.Toe_z', 'acc_L.Heel_x', 'acc_L.Heel_z'
            , 'acc_Left_MTP_lat_y', 'acc_Left_MTP_lat_z', 'acc_Left_cal_lat_y', 'acc_Left_cal_lat_z']

cutoff_freq = 20  # Hz
order = 4
sampling_rate = 60  # Assuming the sampling rate is 60 Hz, change it to the actual value


#df.sort_index().interpolate(method='spline', order=3, inplace=True)
    
    
filtered_feature = pd.DataFrame()  # Create an empty list to store the filtered data

for feature in features:
    data = acc_df[feature]  # assuming acc_df is the original DataFrame containing your data
    filtered_data = butterworth_filter(data, cutoff_freq, order, sampling_rate)
    filtered_feature[feature] = filtered_data

In [ ]:
filtered_df = filtered_feature

In [ ]:
filtered_df = filtered_df.reset_index(drop=True)
acc_df2 = acc_df2.reset_index(drop=True)
filtered_df['PAD'] = acc_df2['PAD']
filtered_df['Subject'] = acc_df2['Subject']
filtered_df['Subject'].nunique()

In [ ]:
# Splitting the dataset subject based. 40 controls devided by the ratio of 60%, 20% and 20%. 73 PAD: 43 train, 15, 15.

train = filtered_df[filtered_df['Subject'].isin(['s10', 's11', 's16', 's17', 's18', 's19', 's20', 's21', 's22',
                                   's23', 's24', 's25', 's13', 's14', 's26', 's29', 's36 ', 's500',
                                   's49', 's50', 's51', 's52', 's54', 's56',
                                   's119', 's122', 's134', 's137', 's138', 's322', 's327', 's333', 
                                   's335', 's340', 's341', 's342', 's343', 's344', 's345', 's346', 
                                   's347', 's348', 's349', 's350', 's352','s136', 's361', 's362', 
                                   's364', 's366', 's368', 's370', 's371', 's372', 's432', 's433', 
                                   's436', 's440', 's441', 's444', 's445','s487', 's488', 's503', 
                                   's507', 's513', 's527'])]
valid = filtered_df[filtered_df['Subject'].isin(['s31', 's33', 's34', 's35' , 's53', 's55', 's57', 's12',
                                   's353', 's354', 's356', 's359', 's363', 's367', 's369', 's373',
                                   's469', 's473', 's477', 's482', 's483', 's484', 's528'])]
test = filtered_df[filtered_df['Subject'].isin(['s37', 's39', 's40', 's41', 's43', 's48', 's38', 's45',
                                  's374', 's375', 's376', 's377', 's379', 's121', 's126', 's131',
                                  's450', 's457', 's458', 's459', 's460', 's462', 's466'])]

In [ ]:
# Some checks
print(train['Subject'].nunique())
print(valid['Subject'].nunique())
print(test['Subject'].nunique())
all_subjects = acc_df['Subject'].unique()
assert set(train['Subject'].unique()) | set(valid['Subject'].unique()) | set(test['Subject'].unique()) == set(all_subjects)

In [ ]:
#Shuffling the data on the subjects

# Create a list of unique subject IDs
unique_subject_ids = train['Subject'].unique()

# Shuffle the order of the subject IDs
shuffled_subject_ids = shuffle(unique_subject_ids, random_state=42)

# Create a new DataFrame for the shuffled data
shuffled_tr = pd.DataFrame(columns=train.columns)

# Iterate over the shuffled subject IDs and append the corresponding data to the new DataFrame
for subject_id in shuffled_subject_ids:
    subject_data = train[train['Subject'] == subject_id]
    shuffled_tr = shuffled_tr.append(subject_data)

# Reset the index of the shuffled DataFrame
shuffled_tr.reset_index(drop=True, inplace=True)

In [ ]:
unique_subject_ids = valid['Subject'].unique()

shuffled_subject_ids = shuffle(unique_subject_ids, random_state=42)

shuffled_va = pd.DataFrame(columns=valid.columns)

for subject_id in shuffled_subject_ids:
    subject_data = valid[valid['Subject'] == subject_id]
    shuffled_va = shuffled_va.append(subject_data)

shuffled_va.reset_index(drop=True, inplace=True)

In [ ]:
unique_subject_ids = test['Subject'].unique()

shuffled_subject_ids = shuffle(unique_subject_ids, random_state=42)

shuffled_te = pd.DataFrame(columns=test.columns)

for subject_id in shuffled_subject_ids:
    subject_data = test[test['Subject'] == subject_id]
    shuffled_te = shuffled_te.append(subject_data)

shuffled_te.reset_index(drop=True, inplace=True)

In [ ]:
tain_sub = shuffled_tr['Subject']
val_sub = shuffled_va['Subject']
test_sub = shuffled_te['Subject']
shuffled_tr.drop(['Subject'], axis = 1, inplace = True)
shuffled_va.drop(['Subject'], axis = 1, inplace = True)
shuffled_te.drop(['Subject'], axis = 1, inplace = True)

In [ ]:
cols = ['acc_R.ASIS_x', 'acc_Right_PSIS_x', 'acc_Right_PSIS_y', 'acc_Right_PSIS_z', 'acc_L.ASIS_x', 'acc_L.ASIS_y'
        , 'acc_L.ASIS_z', 'acc_Left_PSIS_x', 'acc_Left_PSIS_y', 'acc_Left_PSIS_z', 'acc_Right_Troch_x', 'acc_Right_Troch_y'
        , 'acc_Right_Troch_z', 'acc_Right_mid_thigh_x', 'acc_Right_mid_thigh_y', 'acc_Right_mid_thigh_z', 'acc_R.Thigh_x'
        , 'acc_R.Thigh_y', 'acc_R.Thigh_z', 'acc_R.Knee_x', 'acc_R.Knee_y', 'acc_R.Knee_z', 'acc_R.Shank_x', 'acc_R.Shank_y'
        , 'acc_R.Shank_z', 'acc_Right_lower_shank_x', 'acc_Right_lower_shank_y', 'acc_Right_lower_shank_z', 'acc_R.Ankle_y'
        , 'acc_R.Toe_x', 'acc_R.Toe_y', 'acc_R.Toe_z', 'acc_R.Heel_x', 'acc_R.Heel_y', 'acc_R.Heel_z', 'acc_Right_MTP_lat_y'
        , 'acc_Right_cal_lat_y', 'acc_Left_Troch_x', 'acc_Left_Troch_y', 'acc_Left_Troch_z', 'acc_Left_mid_thigh_x'
        , 'acc_L.Thigh_y', 'acc_L.Thigh_z', 'acc_L.Knee_y', 'acc_Left_lower_shank_y', 'acc_Left_lower_shank_z'
        , 'acc_L.Ankle_x', 'acc_L.Ankle_y', 'acc_L.Toe_x', 'acc_L.Toe_y', 'acc_L.Toe_z', 'acc_L.Heel_x', 'acc_L.Heel_z'
        , 'acc_Left_MTP_lat_y', 'acc_Left_MTP_lat_z', 'acc_Left_cal_lat_y', 'acc_Left_cal_lat_z']

Xa = shuffled_tr[cols]
ya = shuffled_tr['PAD']

Xv = shuffled_va[cols]
yv = shuffled_va['PAD']

Xt = shuffled_te[cols]
yt = shuffled_te['PAD']

In [ ]:
for cols in Xa.select_dtypes(include=['float64']).columns:
    Xa[cols] = Xa[cols].astype('float32')
    
for cols in Xv.select_dtypes(include=['float64']).columns:
    Xv[cols] = Xv[cols].astype('float32')
    
for cols in Xt.select_dtypes(include=['float64']).columns:
    Xt[cols] = Xt[cols].astype('float32')

In [ ]:
all_finite = Xa.all().all()
print("All values are finite:", all_finite)

In [ ]:
le = LabelEncoder()
scaler = StandardScaler()

In [ ]:
ya = le.fit_transform(ya)
yv = le.transform(yv)
yt = le.transform(yt)

Xa = scaler.fit_transform(Xa)
Xv = scaler.transform(Xv)
Xt = scaler.transform(Xt)

In [ ]:
print(Xa.shape)
print(Xv.shape)
print(Xt.shape)

In [ ]:
num_steps = 100
def lstm_data_transform(x_data, y_data, num_steps= num_steps):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps
        # if index is larger than the size of the dataset, we stop
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        # Get only the last element of the sequency for y
        seq_y = y_data[end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

In [ ]:
Xa = np.array(Xa)
Xv = np.array(Xv)
Xt = np.array(Xt)

ya = np.array(ya)
yv = np.array(yv)
yt = np.array(yt)

In [ ]:
num_steps = 100
# training set
(Xa, ya) = lstm_data_transform(Xa, ya, num_steps=num_steps)
assert Xa.shape[0] == ya.shape[0]
# validation set
(Xv, yv) = lstm_data_transform(Xv, yv, num_steps=num_steps)
assert Xv.shape[0] == yv.shape[0]
# test set
(Xt, yt) = lstm_data_transform(Xt, yt, num_steps=num_steps)
assert Xt.shape[0] == yt.shape[0]

In [ ]:
import optuna
import datetime
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

# Assuming Xa, Xv, Xt, ya, yv, yt are already defined elsewhere

def objective(trial):

    # Define hyperparameters to be tuned
    lr = trial.suggest_float("lr", 1e-4, 1e-3, log=True)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    lstm_units = trial.suggest_int("lstm_units", 2, 128, log=True)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16, 32])
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "rmsprop"])

    # Model definition
    model = Sequential()
    model.add(LSTM(lstm_units, activation='tanh', input_shape=(100, 57), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(lstm_units, activation='tanh', return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(lstm_units, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))

    # Choose optimizer
    if optimizer_name == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=lr)
    elif optimizer_name == "rmsprop":
        optimizer = keras.optimizers.RMSprop(learning_rate=lr)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    es_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=3, restore_best_weights=True)

    # Train the model
    history = model.fit(Xa, ya, validation_data=(Xv, yv), epochs=1, batch_size=batch_size, callbacks=[es_cb], verbose=1)

    # Return the best validation accuracy
    val_acc = max(history.history['val_accuracy'])
    return val_acc

print('training is started!')
print(datetime.datetime.now())
# Start the Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)


# Print out the results
print(datetime.datetime.now())
print(str(study))

print(f"Number of finished trials: {len(study.trials)}")
print(f"Best trial: {study.best_trial.params}")

In [ ]:
def earlystopping(min_delta, patience):
    es_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                             min_delta=min_delta,
                                             patience=patience,
                                             restore_best_weights=True)
    return es_cb


num_steps = 100
num_features = 57
max_epochs = 1
patience = 5
min_delta = 1e-5
batch_size = 32
Beta1 = 0.9
Beta2 = 0.999
Epsilon = 10^-8
optimizer = 'adam'
best_lr = 0.001

model_16 = Sequential()
#model_2.add(LSTM(64, activation='relu', input_shape=(num_steps, num_features), return_sequences=True))
#model_2.add(Dropout(0.2))
#model_2.add(LSTM(units = 32, activation='relu', return_sequences = True))
#model_2.add(Dropout(0.5))
#model_2.add(LSTM(units = 16, activation='relu', return_sequences = True))
#model_2.add(Dropout(0.5))
model_16.add(LSTM(units = 4, activation='tanh', dropout=0.5, recurrent_dropout=0.5, return_sequences = True))
model_16.add(Dropout(0.7))
model_16.add(LSTM(units = 4, activation='tanh', dropout=0.5, recurrent_dropout=0.5, return_sequences = True))
model_16.add(Dropout(0.5))
model_16.add(LSTM(units = 4, activation='tanh', dropout=0.5, recurrent_dropout=0.5, return_sequences = True))
model_16.add(Dropout(0.5))
model_16.add(LSTM(units = 4, activation='tanh', dropout=0.5, recurrent_dropout=0.5, return_sequences = False))
model_16.add(Dense(1, activation='sigmoid'))

if optimizer == "adam":
    optimizer = tf.keras.optimizers.Adam(learning_rate=best_lr)
else:  # It's redundant here since we know the best optimizer is Adam, but it's good to have for completeness
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=best_lr)
    

# compile
model_16.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  optimizer=optimizer,
                  metrics=['accuracy'])
    

# train
model_16_history = model_16.fit(Xa, ya,
                        epochs=max_epochs,
                        validation_data=(Xv, yv),
                        shuffle=False,
                        callbacks=[earlystopping(min_delta, patience)],
                        batch_size=batch_size)

In [ ]:
# Evaluate the model on the test data
loss, accuracy = model_16.evaluate(Xt, yt, verbose=0)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Testing the model
test_predict = model_16.predict(Xt)
y_pred = (test_predict > 0.5)

In [ ]:
# Confusion Matrix
import seaborn as sn
cm = confusion_matrix(yt, y_pred)
print(cm)
accuracy_score(yt, y_pred)

df_cm = pd.DataFrame(cm, range(2), range(2))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
plt.title("Accuracy: %.2f%%" % (accuracy_score(yt, y_pred)*100))
plt.xlabel("Predictions")
plt.ylabel("Actual")
plt.show()

In [ ]:
from keras.backend import repeat
final = pd.DataFrame(np.concatenate((y_pred.reshape(len(y_pred),1), yt.reshape(len(yt),1)),1))
final.columns = ['Predict','Actual']

final = final.reset_index()

sub = pd.DataFrame(test_sub[100:]).reset_index(drop=True)

final1 = pd.concat([final, sub], axis=1)

final1['pred'] = final1.groupby('Subject')['Predict'].transform('mean')
final1['prediction_result'] = (final1.pred > 0.5)
final1['Prediction'] = final1['prediction_result'].astype(int)

In [ ]:
final1 = final1[['Subject', 'Prediction', 'Actual']]
final1 = final1.groupby(['Subject'], as_index = False).mean()

In [ ]:
yt1 = final1['Actual']
yp = final1['Prediction']
cm1 = confusion_matrix(yt1, yp)
print(cm1)
accuracy_score(yt1, yp)

df_cm1 = pd.DataFrame(cm1, range(2), range(2))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm1, annot=True, annot_kws={"size": 16}) # font size
plt.title("Accuracy: %.2f%%" % (accuracy_score(yt1, yp)*100))
plt.xlabel("Predictions")
plt.ylabel("Actual")
plt.show()